# Imports<br>
I used Kaggle with GPU T4 X2 for this Notebook

In [ ]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f2f2535f-8031-c2e3-cbbd-7ac821018a19)

GPU 1: Tesla T4 (UUID: GPU-ed4446fe-ef80-3b3d-92ed-52ec7fa2bde7)


In [ ]:
%%time

from IPython.display import clear_output

! pip install -qq -U langchain
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu

! pip install sentence_transformers==2.2.2
! pip install -qq -U InstructorEmbedding

! pip install -qq -U transformers
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes
! pip install langchain_community

clear_output()

CPU times: user 1.95 s, sys: 528 ms, total: 2.48 s

Wall time: 2min 46s


In [ ]:
%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time
import gc
import json

import langchain
import pandas as pd

### loaders
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import DataFrameLoader

### splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

### prompts
from langchain import PromptTemplate

### vector stores
from langchain_community.vectorstores import FAISS

### models
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

### retrievers
from langchain.chains import RetrievalQA

import torch

import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

clear_output()

CPU times: user 10.7 s, sys: 1.79 s, total: 12.5 s

Wall time: 18 s


In [ ]:
print('langchain:', langchain.__version__)
print('torch:', torch.__version__)
print('transformers:', transformers.__version__)

langchain: 0.2.0

torch: 2.1.2

transformers: 4.41.0


# Configurations

In [ ]:
class CFG:
    DEBUG = False

    # LLM
    model_name = 'microsoft/Phi-3-mini-128k-instruct'
    temperature = 0.4
    top_p = 0.90
    repetition_penalty = 1.15
    max_len = 8192
    max_new_tokens = 512

    # splitting
    split_chunk_size = 800
    split_overlap = 400

    # embeddings
    embeddings_model_repo = 'BAAI/bge-base-en-v1.5'

    # similar passages
    k = 6

    # paths
    PDFs_path = '/kaggle/input/100-llm-papers-to-explore/'
    Embeddings_path =  '/kaggle/input/faiss-ml-papers-st'
    Output_folder = './ml-papers-vectordb'
    Relations_path = "/kaggle/input/attention-ai/relationships_minimized.json"
    DataModel = "/kaggle/input/attention-ai/DataModel"

# Loading Data

In [ ]:
try:
    with open(CFG.Relations_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    print("The file was not found.")
except json.JSONDecodeError:
    print("Error decoding the JSON data.")
except Exception as e:
    print(f"An error occurred: {e}")

relationships = []
for key, value in data.items():
    for relation in value:
        relationships.append({
            "FromTable": relation["FromTable"],
            "FromColumn": ", ".join(relation["FromColumn"]),
            "ToTable": relation["ToTable"],
            "ToColumn": ", ".join(relation["ToColumn"])
        })
df = pd.DataFrame(relationships)
df.to_csv('relationships.csv', index=False)

In [ ]:
json_files = []
def list_json_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                json_files.append(os.path.join(root, file))
    return json_files

def list_directories(directory):
    directories = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    return directories

def main(directory):
    directories = list_directories(directory)
    for dir_path in directories:
        print(f"Files in directory: {dir_path}")
        json_files = list_json_files(dir_path)
        for json_file in json_files:
            print(json_file)

if __name__ == "__main__":
    main(CFG.DataModel)
path_dict = {path.split('/')[-1].rsplit('.', 1)[0]: path for path in json_files}
clear_output()

In [ ]:
def normalize_keys(d):
    try:
        return {k.lower(): v for k, v in d.items()}
    except AttributeError:
        return {}

def get_row_info(row):
    FromTable_path = path_dict.get(row['FromTable'], '')
    ToTable_path = path_dict.get(row['ToTable'], '')
    try:
        with open(FromTable_path, 'r') as FromTable_path_file:
            FromTable = json.load(FromTable_path_file)
        with open(ToTable_path, 'r') as ToTable_path_file:
            ToTable = json.load(ToTable_path_file)

        FromTable = normalize_keys(FromTable)
        ToTable = normalize_keys(ToTable)

        FromTable_desc = FromTable.get('description', '')
        ToTable_desc = ToTable.get('description', '')

        FromColumn_desc = {}
        ToColumn_desc = {}

        if 'columns' in FromTable:
            for column in FromTable['columns']:
                FromColumn_desc[column.get('name')] = column.get('description', '')

        if 'columns' in ToTable:
            for column in ToTable['columns']:
                ToColumn_desc[column.get('name')] = column.get('description', '')

        return FromTable_desc, FromColumn_desc, ToTable_desc, ToColumn_desc

    except (FileNotFoundError, json.JSONDecodeError, KeyError):
        return '', '', '', ''


In [ ]:
df[['FromTable_desc', 'FromColumn_desc', 'ToTable_desc', 'ToColumn_desc']] = df.apply(lambda row: pd.Series(get_row_info(row)), axis=1)

In [ ]:
def format_row_info(row):
    formatted_info1 = ''
    formatted_info2 = ''
#     from_column_desc = ast.literal_eval(row['FromColumn_desc'])
#     to_column_desc = ast.literal_eval(row['ToColumn_desc'])

    for key, value in dict(row['FromColumn_desc']).items():
        formatted_info1 += f"- {key}: {value}\n"
    for key, value in dict(row['ToColumn_desc']).items():
        formatted_info2 += f"- {key}: {value}\n"
    formatted_string = (f"There is a relationship between data tables '{row['FromTable']}' and '{row['ToTable']}' with the common column '{row['FromColumn']}'\n"
       f"the table '{row['FromTable']}' contains columns and column-description as {formatted_info1} and the table '{row['ToTable']}' contains columns and column-description as {formatted_info2} respectively.\n")
    return formatted_string

In [ ]:
df[['relationship']] = df.apply(lambda row: pd.Series(format_row_info(row)), axis=1)
df.drop(['FromTable_desc','FromColumn_desc','ToTable_desc','ToColumn_desc'], axis = 1, inplace = True)

In [ ]:
loader = DataFrameLoader(df , page_content_column = 'relationship')
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)

texts = text_splitter.split_documents(documents)

print(f'We have created {len(texts)} chunks from {len(documents)} pages')

We have created 275 chunks from 52 pages


# Vector Database

In [ ]:
%%time

### we create the embeddings if they do not already exist in the input folder
if not os.path.exists(CFG.Embeddings_path + '/index.faiss'):

    print('Creating embeddings...\n\n')

    ### download embeddings model
    embeddings = HuggingFaceInstructEmbeddings(
        model_name = CFG.embeddings_model_repo,
        model_kwargs = {"device": "cuda"}
    )

    ### create embeddings and DB
    vectordb = FAISS.from_documents(
        documents = texts,
        embedding = embeddings
    )

    ### persist vector database
    vectordb.save_local(f"{CFG.Output_folder}/faiss_index_ml_tables") # save in output folder
#     vectordb.save_local(f"{CFG.Embeddings_path}/faiss_index_ml_papers") # save in input folder

clear_output()

CPU times: user 6.13 s, sys: 2.38 s, total: 8.52 s

Wall time: 14.9 s


In [ ]:
%%time

### download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
)

### load vector DB embeddings
vectordb = FAISS.load_local(
#     CFG.Embeddings_path, # from input folder
    CFG.Output_folder + '/faiss_index_ml_tables', # from output folder
    embeddings,
    allow_dangerous_deserialization = True,
)

clear_output()

CPU times: user 243 ms, sys: 221 ms, total: 464 ms

Wall time: 283 ms


In [ ]:
%%time

### test if vector DB was loaded correctly
vectordb.similarity_search('Give me top 5 customer with most number of orders.')
clear_output()

CPU times: user 124 ms, sys: 5.66 ms, total: 129 ms

Wall time: 135 ms


# PHI3 using HuggingFace Pipeline

In [ ]:
def build_model(model_repo = CFG.model_name):

    print('\nDownloading model: ', model_repo, '\n\n')

    ### tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_repo)

    ### quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.float16,
        bnb_4bit_use_double_quant = True,
    )

    ### model
    model = AutoModelForCausalLM.from_pretrained(
        model_repo,
        quantization_config = bnb_config,
        device_map = 'auto',
        low_cpu_mem_usage = True,
        trust_remote_code = True,
    )

    return tokenizer, model

In [ ]:
%%time

tokenizer, model = build_model(model_repo = CFG.model_name)

clear_output()

CPU times: user 18.9 s, sys: 15.8 s, total: 34.7 s

Wall time: 45.8 s


In [ ]:
model.eval()

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3SuScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features

In [ ]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.bos_token_id
]


### hugging face pipeline
pipe = pipeline(
    task = "text-generation",

    model = model,

    tokenizer = tokenizer,
#     pad_token_id = tokenizer.eos_token_id,
    eos_token_id = terminators,

    do_sample = True,
#     max_length = CFG.max_len,
    max_new_tokens = CFG.max_new_tokens,


    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty,
)

### langchain pipeline
llm = HuggingFacePipeline(pipeline = pipe)
clear_output()

In [ ]:
prompt_template = """
<|system|>
You're a helpful assistant who can recommend data tables and columns to help solve a given problem. You have information about various tables, including their descriptions and column details.

If there isn't an exact answer available, you'll suggest the closest matching tables and columns.
Remember to respond in the same language as the question

<|end|>

<|user|>

Context: {context}

Question is below.
Question: {question}

<|end|>

<|assistant|>

"""
prompt_template = """
<|system|>

You're a helpful assistant who can recommend data tables and columns to help solve a given problem.
You have information about various tables, including their descriptions and column details.
Give proper reason of your choise of tables and columns that you have choosen to solve the problem.

If there isn't an exact answer available, you'll suggest the closest matching tables and columns.
final answer should be in form 'FromTable': first data table name , 'Column': Common Column name Common to both data tables, 'ToTable': Second data table name.
Remember to respond in the same language as the question.
<|end|>

<|user|>

Context: {context}

Question is below.
Question: {question}
<|end|>

<|assistant|>
"""

PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [ ]:
retriever = vectordb.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": CFG.k}
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever,
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

# Generation and formating output

In [ ]:
def wrap_text_preserve_newlines(text, width=1500):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate_format_output(query, resoning = False):
    llm_response = qa_chain.invoke(query)
    ans = wrap_text_preserve_newlines(llm_response['result'])
    pattern = "<|assistant|>"
    index = ans.find(pattern)
    if index != -1:
        ans = ans[index + len(pattern):]
    if(resoning):
        print(query)
        print("Following are the reasons for selection:\n")
        print("\n")
        print(ans)
    for source in llm_response['source_documents']:
        Relationship = source.metadata
    relation = f"#TableRelationship \n'relationships': [\
{{\
'FromTable': {Relationship['FromTable']},\
'FromColumn': [\
{Relationship['FromColumn']}\
],\
'ToTable': {Relationship['ToTable']},\
'ToColumn': [\
{Relationship['ToColumn']}\
]\
}}\
]\n"
    FromTable_path = path_dict.get(Relationship['FromTable'], '')
    ToTable_path = path_dict.get(Relationship['ToTable'], '')
    with open(FromTable_path, 'r') as FromTable_path_file:
        FromTable = json.load(FromTable_path_file)
    with open(ToTable_path, 'r') as ToTable_path_file:
        ToTable = json.load(ToTable_path_file)
    print("RequiredColumns\n")
    print(FromTable)
    print("\n")
    print(ToTable)
    print("\n")
    print(relation)


In [ ]:
query = "Can you demonstrate how to compute the monthly percentage change in revenue for each category of products, comparing it with the prior month's performance?"
generate_format_output(query)

RequiredColumns



{'name': 'product_category_pricing', 'description': 'Stores product pricing across categories', 'primary_key_column': ['product_id', 'category_id'], 'clustering_column': ['category_id', 'brand_id'], 'partitioning_column': ['date_updated'], 'type': 'Type 2 Dimension Table', 'columns': [{'name': 'product_id', 'description': 'Unique identifier for products', 'data_type': 'STRING', 'format': '', 'is_pii_column': 'N', 'enum': [], 'dimension_group': ''}, {'name': 'category_id', 'description': 'Identifier for product category', 'data_type': 'STRING', 'format': '', 'is_pii_column': 'N', 'enum': [], 'dimension_group': 'category_dimension_group'}, {'name': 'brand_id', 'description': 'Identifier for product brand', 'data_type': 'STRING', 'format': '', 'is_pii_column': 'N', 'enum': [], 'dimension_group': 'product_dimension_group'}, {'name': 'price', 'description': 'Current price of the product', 'data_type': 'FLOAT', 'format': '', 'is_pii_column': 'N', 'enum': [], 'dimension_gro

In [ ]:
query = "Can you identify the geographic regions where the total annual revenue last year exceeded the combined average of all regions, and also provide the percentage by which they exceeded this average?"
generate_format_output(query)

RequiredColumns



{'name': 'operational_metrics_fact', 'description': 'Stores daily operational metrics for analysis', 'primary_key_column': ['date', 'store_id'], 'clustering_column': ['store_id', 'product_id'], 'partitioning_column': ['date'], 'type': 'Type 1 Fact Table', 'Columns': [{'name': 'date', 'description': 'The date of data recording', 'data_type': 'DATE', 'format': 'YYYY-MM-DD', 'is_pii_column': 'N', 'enum': []}, {'name': 'store_id', 'description': 'Unique identifier for a store', 'data_type': 'STRING', 'format': '', 'is_pii_column': 'N', 'enum': []}, {'name': 'product_id', 'description': 'Unique identifier for a product', 'data_type': 'STRING', 'format': '', 'is_pii_column': 'N', 'enum': []}, {'name': 'sales_volume', 'description': 'Total sales volume for the product', 'data_type': 'INTEGER', 'format': '', 'is_pii_column': 'N', 'enum': []}, {'name': 'sales_amount', 'description': 'Total sales amount in USD', 'data_type': 'FLOAT', 'format': '', 'is_pii_column': 'N', 'enum':

Reference for my assignment: [RAG with PHI](https://www.kaggle.com/code/hinepo/rag-with-phi-3). <br>
I used RAG with phi3 to handle the DataMode and Table relationships, as fine-tuning isn't feasible with only questions provided.